In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [11]:
import os
import sys

base_dir = "/content/gdrive/MyDrive/Workspace/Colab"
sys.path.append(base_dir)

src_dir = os.path.join(base_dir, "langchain")
%cd $src_dir

/content/gdrive/MyDrive/Workspace/Colab/langchain


In [12]:
!pip install -r requirements.txt > /dev/null

## **Model**

In [13]:
import import_ipynb
from utils.decorator import *

In [14]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SERP_API_KEY = os.getenv("SERP_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("Google Search API key not found in .env file.")
if not SERP_API_KEY:
    raise ValueError("OpenAI API key not found in .env file.")

In [15]:
from langchain_community.llms import OpenAI
from langchain.callbacks import get_openai_callback
from langchain.globals import set_llm_cache, set_debug
from langchain_community.cache import RedisCache, InMemoryCache, RedisSemanticCache
from langchain.embeddings import OpenAIEmbeddings

In [16]:
cache = InMemoryCache()

cache.clear()
set_llm_cache(cache)

* temperature: 모델 출력의 창의성, 무작위성 조정 (확률분포의 분산 조절)
* top_p: 누적확률샘플링 기반, 모델이 단어를 선택할 확률 범위 제한

In [17]:
class TestModel:

    def __init__(self):
        self.model = OpenAI(
            api_key=OPENAI_API_KEY,
            temperature=0.0,
            top_p=1.0,
            n=2,
            best_of=2,
            cache=True
            )

    @elapsed_time
    def get_response_sync(self, prompt: str) -> str:
        return self._invoke_model(prompt)

    @elapsed_time
    async def get_response_async(self, prompt: str) -> str:
        return await self._invoke_model_async(prompt)

    def _invoke_model(self, prompt: str) -> str:
        try:
            return self.model.invoke(prompt)
        except Exception as e:
            raise ValueError(f"모델 호출 중 오류 발생: {str(e)}")

    async def _invoke_model_async(self, prompt: str) -> str:
        try:
            return await self.model.ainvoke(prompt)
        except Exception as e:
            raise ValueError(f"비동기 모델 호출 중 오류 발생: {str(e)}")

In [18]:
import json


def _get_meta_info(callback) -> dict:

    return {
        "Tokens Used": callback.total_tokens,
        "Prompt Tokens": callback.prompt_tokens,
        "Completion Tokens": callback.completion_tokens,
        "Total Cost (USD)": callback.total_cost,
        "Successful Requests": callback.successful_requests,
    }


def _convert_to_pretty_json(data: dict) -> str:
    return json.dumps(data, indent=4)

In [19]:
llm = TestModel()

<ipython-input-17-117291d638e2>:4: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  self.model = OpenAI(


In [20]:
prompt = "What is famous street foods in Seoul Korea in 200 characters"

with get_openai_callback() as cb:

    response = llm.get_response_sync(prompt)
    print(_convert_to_pretty_json(_get_meta_info(cb)))

# Caching
with get_openai_callback() as cb:

    response = llm.get_response_sync(prompt)
    print(_convert_to_pretty_json(_get_meta_info(cb)))

Sync execution time: 1.4620064089999687
{
    "Tokens Used": 236,
    "Prompt Tokens": 12,
    "Completion Tokens": 224,
    "Total Cost (USD)": 0.000466,
    "Successful Requests": 1
}
Sync execution time: 0.0005176779999942482
{
    "Tokens Used": 0,
    "Prompt Tokens": 0,
    "Completion Tokens": 0,
    "Total Cost (USD)": 0.0,
    "Successful Requests": 0
}


In [21]:
import tiktoken


def _get_tokens(text: str, model_name: str = "gpt-4") -> int:
    """
    tokenizer
    """
    encoding = tiktoken.encoding_for_model(model_name)
    return encoding.encode(text)

In [22]:
tokens = _get_tokens(prompt, model_name="gpt-3.5-turbo")

print(f"Token count: {len(tokens)}")
print(f"Tokens: {tokens}")

Token count: 12
Tokens: [3923, 374, 11495, 8761, 15657, 304, 51289, 12126, 304, 220, 1049, 5885]


*Tracking*

In [23]:
from langchain.agents import load_tools, initialize_agent, AgentType


tools = load_tools(["serpapi"], serpapi_api_key=SERP_API_KEY)
agent = initialize_agent(
    tools,
    llm.model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

with get_openai_callback() as cb:

    response = agent.run(
        prompt
    )
    print(_convert_to_pretty_json(_get_meta_info(cb)))

print()

with get_openai_callback() as cb:

    response = agent.run(
        prompt
    )
    print(_convert_to_pretty_json(_get_meta_info(cb)))

<ipython-input-23-9cb5f974d499>:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(
<ipython-input-23-9cb5f974d499>:15: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(




> Entering new AgentExecutor chain...
 I should always think about what to do
Action: Search
Action Input: "famous street foods in Seoul Korea"
Observation: {'type': 'organic_result'}
Thought: I should click on the first link
Action: Search
Action Input: "famous street foods in Seoul Korea"
Observation: {'type': 'organic_result'}
Thought: I should read the article and take notes
Action: Search
Action Input: "famous street foods in Seoul Korea"
Observation: {'type': 'organic_result'}
Thought: I should look for a list of top street foods
Action: Search
Action Input: "famous street foods in Seoul Korea"
Observation: {'type': 'organic_result'}
Thought: I should look for a list of top street foods in Seoul
Action: Search
Action Input: "top street foods in Seoul Korea"
Observation: {'type': 'organic_result'}
Thought: I should read the article and take notes
Action: Search
Action Input: "top street foods in Seoul Korea"
Observation: {'type': 'organic_result'}
Thought: I should look for a li